In [1]:
from nltk import toolbox as tb
from xml.etree import ElementTree as ET

# use NLTK to parse toolbox data as an XML tree
td = tb.ToolboxData()
td.open('tobelo.db')
tob = td.parse()
td.close()


print('Main entries (lx): ' , len(tob)-1)
    
n=0
for entry in tob:
    n += len(entry.findall('se'))
print('Subentries (se): ' , n)
    


Main entries (lx):  3237
Subentries (se):  14


In [19]:
tob[1][0].tag, tob[1][0].text

('lx', 'abari')

In [36]:
entry = tob[22]
for se in entry.findall('se'):
    print(se.text)
print(len(entry.findall('ge')))

o akere ma utu
o akere ma biono
o akere ma jobubu
o huhu ma akere
o akere ma lako
6


In [5]:
for entry in tob[:10]:
    lx = entry.findtext('lx')
    ps = entry.findtext('ps')
    defn = entry.findtext('ge')
    print([lx,ps,defn])

[None, None, None]
['abari', 'o', 'news']
['abo', 'ma', 'foam']
['ade-ade', 'ho', 'to narrate']
['adono', 'ho', 'come up to']
['aerani', 'ma', 'wonderful']
['aere', 'i na', 'ant trails']
['aere', 'o', 'high *tide']
['aga', 'ho', 'dumb']
['agomo', 'ma', 'thick']


In [7]:
# write out first entries as csv
import csv

entries = []
n = 0
for entry in tob[1:]:
    n += 1
    lx = entry.findtext('lx')
    ps = entry.findtext('ps')
    if entry.findtext('de') == None:
        ges = []
        for ge in entry.findall('ge'):
            ges.append(ge.text)
        de = ', '.join(ges)
    else:
        de = entry.findtext('de')
    if entry.findtext('sd') == None:
        sd = ''
    else:
        sd = entry.findtext('sd')
    entries.append([n, lx,ps,de,sd])


with open("tob.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(entries)
    